# HIFI-GAN synthesis

In [ ]:
# Code: https://github.com/jik876/hifi-gan

In [1]:
import torch
import time
import os
import simpleaudio as sa
import librosa
import soundfile
import numpy as np
import matplotlib
matplotlib.use('TkAgg', force=True)
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import pdb
import time

sys.path.insert(0,"C:\\Programming\\Github\waveglow\\tacotron2")
sys.path.insert(0,"C:\\Programming\\Github\\hifi-gan")
print(sys.path)

['C:\\Programming\\Github\\hifi-gan', 'C:\\Programming\\Github\\waveglow\\tacotron2', 'C:\\Programming\\GitLab\\radarspeech\\speech_synthesis\\scripts', 'C:\\Python39\\python39.zip', 'C:\\Python39\\DLLs', 'C:\\Python39\\lib', 'C:\\Python39', '', 'C:\\Python39\\lib\\site-packages', 'C:\\Python39\\lib\\site-packages\\phonemizer-3.2.1-py3.9.egg', 'C:\\Python39\\lib\\site-packages\\win32', 'C:\\Python39\\lib\\site-packages\\win32\\lib', 'C:\\Python39\\lib\\site-packages\\Pythonwin']


In [2]:
def play_audio(audio_data, fs_Hz):
    ''' Function to convert a numpy array to 16 bit integer array
        and serialize it to play it with simpleaudio.
        Args: 
            audio_data (1d-numpy array): array holding the audio samples.
            fs_Hz (scalar): Sampling frequency.

        Returns: 
            None.
    '''
    audio_data_16bit = audio_data
    nan_indices = np.argwhere(np.isnan(audio_data_16bit))
    if(nan_indices.size > 0):
        print("Error: there are %d NaN values in the waveform. Play aborted." %(nan_indices.size))
        return

    max_value = np.max(abs(audio_data))

    audio_data_16bit = audio_data_16bit*32767/max_value
    audio_data_16bit = audio_data_16bit.astype(np.int16)
    play_obj = sa.play_buffer(audio_data_16bit, 1, 2, fs_Hz)
    play_obj.wait_done()
    
    return audio_data_16bit

## Load the model and inference functions.

In [2]:
# Copied from inference_e2e.py

from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import os
import numpy as np
import argparse
import json
import torch
from scipy.io.wavfile import write
from env import AttrDict
from meldataset import MAX_WAV_VALUE
from models import Generator

h = None
device = None


def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict


def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]


In [ ]:
input_mels_dir = "C:\\Users\\chris\\Documents\\Institut\\corpora\\" \
    + "speech_synthesis\\corpus_six\\S003\\input_params_files\\tacotron_mel_power_spectrogram\\npy"

time_context_frames = 50
fs_hifigan_Hz = 22050

hifi_gan_base_path = "C:\Programming\Github\hifi-gan"
# input_mels_dir = os.path.join(hifi_gan_base_path, "test_mel_files")
# output_dir = os.path.join(hifi_gan_base_path, "generated_files_from_mel")
output_dir = os.path.join(os.getcwd(), "hifi_gan_synthesis_output")
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("Created folder {:s}".format(output_dir))
                          
checkpoint_file = os.path.join(hifi_gan_base_path, "checkpoints\\UNIVERSAL_V1\\g_02500000")

config_file = os.path.join(hifi_gan_base_path, 'config.json')
with open(config_file) as f:
    data = f.read()

json_config = json.loads(data)
h = AttrDict(json_config)

torch.manual_seed(h.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(h.seed)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Instantiate the generator model.
generator = Generator(h).to(device)

state_dict_g = load_checkpoint(checkpoint_file, device)
generator.load_state_dict(state_dict_g['generator'])

filelist = os.listdir(input_mels_dir)

os.makedirs(output_dir, exist_ok=True)

generator.eval()
generator.remove_weight_norm()

# Generate all files.
with torch.no_grad():
    for i, filname in enumerate(filelist):
        x = np.load(os.path.join(input_mels_dir, filname))
        x = torch.FloatTensor(x).to(device)
        
        start_time = time.perf_counter()
        y_g_hat = generator(x)
        audio = y_g_hat.squeeze()
        audio = audio * MAX_WAV_VALUE
        audio = audio.cpu().numpy().astype('int16')
        # plt.plot(audio)
        # plt.show()
        stop_time = time.perf_counter()
        inference_time = stop_time-start_time
        
        print("Mel conversion took {:1.4f} s".format(inference_time))
        print("Real-time factor for wave-glow inference: {:1.3}".format(len(audio)/fs_hifigan_Hz/inference_time))

        output_file = os.path.join(output_dir, os.path.splitext(filname)[0] + '_generated_e2e.wav')
        audio[:160*time_context_frames] = 0 # Remove network settling time.
        write(output_file, fs_hifigan_Hz, audio)
        print(output_file)
print("Finished generation.")
        

In [6]:
input_mels_dir

'C:\\Users\\chris\\Documents\\Institut\\corpora\\speech_synthesis\\corpus_six\\S004\\input_params_files\\tacotron_mel_power_spectrogram\npy'

## Test the HIFI-GAN vocoder on a frame by frame basis.

In [ ]:
input_mels_dir = "C:\\Programming\\GitLab\\radarspeech\\speech_synthesis" \
    + "\\saved_results\\corpus_six\\crn\\phase_mag_db_delta_phase_delta\\run_21_20_44_9\\tacotron_mel_power_spectrogram_predicted_9_21_20_44\\npy"
fs_hifigan_Hz = 22050
time_context_frames = 50

hifi_gan_base_path = "C:\Programming\Github\hifi-gan"
# input_mels_dir = os.path.join(hifi_gan_base_path, "test_mel_files")
# output_dir = os.path.join(hifi_gan_base_path, "generated_files_from_mel")
output_dir = os.path.join(os.getcwd(), "hifi_gan_synthesis_output")
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("Created folder {:s}".format(output_dir))
                          
checkpoint_file = os.path.join(hifi_gan_base_path, "checkpoints\\UNIVERSAL_V1\\g_02500000")

config_file = os.path.join(hifi_gan_base_path, 'config.json')
with open(config_file) as f:
    data = f.read()

json_config = json.loads(data)
h = AttrDict(json_config)

torch.manual_seed(h.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(h.seed)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Instantiate the generator model.
generator = Generator(h).to(device)

state_dict_g = load_checkpoint(checkpoint_file, device)
generator.load_state_dict(state_dict_g['generator'])

filelist = os.listdir(input_mels_dir)

os.makedirs(output_dir, exist_ok=True)

generator.eval()
generator.remove_weight_norm()

# Generate all files.
with torch.no_grad():
    for i, filname in enumerate(filelist):
        x = np.load(os.path.join(input_mels_dir, filname))
        x = torch.FloatTensor(x).to(device)
        
        start_time = time.perf_counter()
        
        # Predict frame by frame.
        frame_by_frame_audio = torch.empty((1,1,1))
        for frame_index in range(x.shape[-1]):
            input_mel_frame = x[0,:,frame_index].unsqueeze(dim=0).unsqueeze(dim=2)
            audio_frame = generator(input_mel_frame)
            frame_by_frame_audio = torch.cat((frame_by_frame_audio, audio_frame), dim=-1)
            
        frame_by_frame_audio = frame_by_frame_audio.squeeze().squeeze()
        frame_by_frame_audio = frame_by_frame_audio * MAX_WAV_VALUE
        frame_by_frame_audio = frame_by_frame_audio.cpu().numpy().astype('int16')  
        
        # predict in one go.
        y_g_hat = generator(x)
        audio = y_g_hat.squeeze()
        audio = audio * MAX_WAV_VALUE
        audio = audio.cpu().numpy().astype('int16')
        
        plt.plot(audio)
        plt.plot(frame_by_frame_audio)
        plt.show()
        
        stop_time = time.perf_counter()
        inference_time = stop_time-start_time
        
        output_file = os.path.join(output_dir, os.path.splitext(filname)[0] + '_generated_e2e.wav')
        output_file_fbf = os.path.join(output_dir, os.path.splitext(filname)[0] + '_generated_e2e_frame_by_frame.wav')
        audio[:160*time_context_frames] = 0 # Remove network settling time.
        write(output_file_fbf, fs_hifigan_Hz, frame_by_frame_audio)
        write(output_file, fs_hifigan_Hz, audio)
        break
        